# 🧮 Phase 2A — Multi-year Bioeconomic LP (NPV)
This notebook extends Phase 1 to **T years** with discounting and time-varying multipliers.

**What’s new:** time index `t=1..T`, NPV objective, scenario arrays.

**What’s NOT yet included (later phases):** crop storage carryover, livestock herd dynamics, risk/uncertainty.


In [4]:
import json
from dataclasses import dataclass
from typing import Dict
from pathlib import Path

import numpy as np
import pandas as pd
from scipy.optimize import linprog

try:
    from caas_jupyter_tools import display_dataframe_to_user
except Exception:
    display_dataframe_to_user = None

DATA = Path('./data_phase_2A')
OUT = Path('./outputs'); OUT.mkdir(exist_ok=True, parents=True)

print('Loaded numpy, pandas, scipy.optimize.linprog')


Loaded numpy, pandas, scipy.optimize.linprog


## Load inputs
Edit the CSVs in `./data/` and the scenario file `scenario_multi.json`.

In [5]:
def must(path: Path) -> Path:
    if not path.exists():
        raise FileNotFoundError(f'Missing required file: {path}')
    return path

hh_df = pd.read_csv(must(DATA/'households.csv'))
crops_df = pd.read_csv(must(DATA/'crops.csv'))
livest_df = pd.read_csv(must(DATA/'livestock.csv'))
prices_df = pd.read_csv(must(DATA/'prices.csv'))
scenario = json.load(open(must(DATA/'scenario_multi.json'),'r'))

obs_path = DATA/'observed_prod_only.csv'
obs_df = pd.read_csv(obs_path) if obs_path.exists() else None

preview = [
    ('households.csv', hh_df),
    ('crops.csv', crops_df),
    ('livestock.csv', livest_df),
    ('prices.csv', prices_df),
    ('scenario_multi.json', pd.DataFrame([{'T':scenario.get('T'), 'discount_rate':scenario.get('discount_rate')}]))
]
for name, df in preview:
    if display_dataframe_to_user: display_dataframe_to_user(name, df)
    else: display(df.head())

if obs_df is not None:
    if display_dataframe_to_user: display_dataframe_to_user('observed_prod_only.csv', obs_df)
    else: display(obs_df.head())

print('Loaded inputs.')


,name,n_households,adult_equiv,labor_endowment,land_available,max_hired_labor
0,HFR,100,3.8,220,0.8,120
1,HMR,100,3.9,240,1.0,130
2,MFIR,100,4.2,300,1.2,160
3,MMR,100,4.2,300,1.2,160
4,MMIR,100,4.6,320,1.5,180


,name,calorie_per_kg,yield_per_ha,price_sale,seed_cost_per_ha,fert_cost_per_ha,chem_cost_per_ha,labor_req_per_ha
0,maize,3600,3500,5.0,800,1800,600,60
1,beans,3400,1500,8.0,900,700,400,70


,name,price_sale,feed_cost_per_unit,vet_cost_per_unit,labor_req_per_unit
0,goat,2500,800,200,6.0
1,chicken,300,60,15,0.6


,wage
0,80


,T,discount_rate
0,10,0.1


,household_class,rev_crops,rev_livestock,off_farm_labor,cost_crop_inputs,cost_livestock,cost_hired_labor
0,HFR,1726.28,844.47,463.33,1166.30,231.00,1195.30
1,HMR,6104.32,1568.32,3649.15,3061.10,70.76,880.50
2,MFIR,5151.81,904.50,1674.58,1431.36,1035.73,992.60
3,MMR,3827.85,1521.11,1376.11,836.60,801.53,721.53
4,MMIR,21985.49,5510.70,3378.98,4247.52,1863.71,1015.24


Loaded inputs.


## Parameter schemas and checks

In [6]:
@dataclass
class HouseholdClass:
    name: str
    n_households: float
    adult_equiv: float
    labor_endowment: float
    land_available: float
    max_hired_labor: float

@dataclass
class CropParam:
    name: str
    calorie_per_kg: float
    yield_per_ha: float
    price_sale: float
    seed_cost_per_ha: float
    fert_cost_per_ha: float
    chem_cost_per_ha: float
    labor_req_per_ha: float

@dataclass
class LivestockParam:
    name: str
    price_sale: float
    feed_cost_per_unit: float
    vet_cost_per_unit: float
    labor_req_per_unit: float

@dataclass
class PriceParam:
    wage: float

@dataclass
class ModelParams:
    households: Dict[str, HouseholdClass]
    crops: Dict[str, CropParam]
    livestock: Dict[str, LivestockParam]
    prices: PriceParam
    min_kcal_per_person_per_day: float = 2000.0
    days_per_year: int = 365

def _req(x, field):
    if pd.isna(x) or str(x).strip()=='':
        raise ValueError(f'Missing value for {field}')
    return float(x)

def load_params() -> ModelParams:
    households = {str(r['name']).strip(): HouseholdClass(
        name=str(r['name']).strip(),
        n_households=_req(r['n_households'],'households.n_households'),
        adult_equiv=_req(r['adult_equiv'],'households.adult_equiv'),
        labor_endowment=_req(r['labor_endowment'],'households.labor_endowment'),
        land_available=_req(r['land_available'],'households.land_available'),
        max_hired_labor=_req(r['max_hired_labor'],'households.max_hired_labor'),
    ) for _, r in hh_df.iterrows()}

    crops = {str(r['name']).strip(): CropParam(
        name=str(r['name']).strip(),
        calorie_per_kg=_req(r['calorie_per_kg'],'crops.calorie_per_kg'),
        yield_per_ha=_req(r['yield_per_ha'],'crops.yield_per_ha'),
        price_sale=_req(r['price_sale'],'crops.price_sale'),
        seed_cost_per_ha=_req(r['seed_cost_per_ha'],'crops.seed_cost_per_ha'),
        fert_cost_per_ha=_req(r['fert_cost_per_ha'],'crops.fert_cost_per_ha'),
        chem_cost_per_ha=_req(r['chem_cost_per_ha'],'crops.chem_cost_per_ha'),
        labor_req_per_ha=_req(r['labor_req_per_ha'],'crops.labor_req_per_ha'),
    ) for _, r in crops_df.iterrows()}

    livestock = {str(r['name']).strip(): LivestockParam(
        name=str(r['name']).strip(),
        price_sale=_req(r['price_sale'],'livestock.price_sale'),
        feed_cost_per_unit=_req(r['feed_cost_per_unit'],'livestock.feed_cost_per_unit'),
        vet_cost_per_unit=_req(r['vet_cost_per_unit'],'livestock.vet_cost_per_unit'),
        labor_req_per_unit=_req(r['labor_req_per_unit'],'livestock.labor_req_per_unit'),
    ) for _, r in livest_df.iterrows()}

    prices = PriceParam(wage=_req(prices_df.iloc[0]['wage'],'prices.wage'))
    return ModelParams(households=households, crops=crops, livestock=livestock, prices=prices)

def _as_T(x, T, name):
    if isinstance(x, (int,float)):
        return [float(x)]*T
    if isinstance(x, list):
        if len(x)!=T:
            raise ValueError(f'scenario.{name} must have length T={T}, got {len(x)}')
        return [float(v) for v in x]
    raise ValueError(f'scenario.{name} must be a number or list length T')

def load_scenario_arrays(sc):
    T = int(sc.get('T', 10))
    r = float(sc.get('discount_rate', 0.10))
    y = _as_T(sc.get('yield_multiplier', 1.0), T, 'yield_multiplier')
    p = _as_T(sc.get('crop_price_multiplier', 1.0), T, 'crop_price_multiplier')
    w = _as_T(sc.get('wage_multiplier', 1.0), T, 'wage_multiplier')
    f = _as_T(sc.get('fert_price_multiplier', 1.0), T, 'fert_price_multiplier')
    pop = _as_T(sc.get('population_multiplier', 1.0), T, 'population_multiplier')
    disc = [1.0/((1.0+r)**t) for t in range(T)]
    return T, r, y, p, w, f, pop, disc

params = load_params()
T, r, Ymul, Pmul, Wmul, Fmul, POPmul, DISC = load_scenario_arrays(scenario)
print(f'Loaded params: H={len(params.households)}, C={len(params.crops)}, L={len(params.livestock)}; T={T}, r={r}')


Loaded params: H=6, C=2, L=2; T=10, r=0.1


## Multi-year LP solver (Phase 2A)
No inter-year storage; each year is linked only via the discounted objective (NPV).

In [7]:
def build_index_maps(H, C, L, T):
    idx = {}; pos = 0
    for t in range(T):
        for h in H:
            for c in C: idx[('area', h, c, t)] = pos; pos += 1
        for h in H:
            for c in C: idx[('cons', h, c, t)] = pos; pos += 1
        for h in H:
            for c in C: idx[('sold', h, c, t)] = pos; pos += 1
        for h in H: idx[('hired', h, None, t)] = pos; pos += 1
        for h in H: idx[('off_farm', h, None, t)] = pos; pos += 1
        for h in H:
            for l in L: idx[('live_units', h, l, t)] = pos; pos += 1
    return idx, pos

def solve_multi_year_lp(params: ModelParams, T, Ymul, Pmul, Wmul, Fmul, POPmul, DISC):
    H = list(params.households.keys())
    C = list(params.crops.keys())
    L = list(params.livestock.keys())
    idx, nvars = build_index_maps(H, C, L, T)
    cvec = np.zeros(nvars)

    for t in range(T):
        disc = DISC[t]
        wage_t = params.prices.wage * Wmul[t]
        for h in H:
            cvec[idx[('hired', h, None, t)]] += disc * wage_t
            cvec[idx[('off_farm', h, None, t)]] += -disc * wage_t
            for cn in C:
                cp = params.crops[cn]
                cvec[idx[('sold', h, cn, t)]] += -disc * (cp.price_sale * Pmul[t])
                per_ha = cp.seed_cost_per_ha + (cp.fert_cost_per_ha * Fmul[t]) + cp.chem_cost_per_ha
                cvec[idx[('area', h, cn, t)]] += disc * per_ha
            for l in L:
                lv = params.livestock[l]
                cvec[idx[('live_units', h, l, t)]] += disc * ((lv.feed_cost_per_unit + lv.vet_cost_per_unit) - lv.price_sale)

    A_eq, b_eq, A_ub, b_ub = [], [], [], []
    bounds = [(0, None) for _ in range(nvars)]

    # Crop balance: yield*area = cons + sold
    for t in range(T):
        for h in H:
            for cn in C:
                row = np.zeros(nvars)
                row[idx[('area', h, cn, t)]] = params.crops[cn].yield_per_ha * Ymul[t]
                row[idx[('cons', h, cn, t)]] = -1.0
                row[idx[('sold', h, cn, t)]] = -1.0
                A_eq.append(row); b_eq.append(0.0)

    # Land
    for t in range(T):
        for h in H:
            row = np.zeros(nvars)
            for cn in C:
                row[idx[('area', h, cn, t)]] = 1.0
            A_ub.append(row); b_ub.append(params.households[h].land_available)

    # Labor: crop + livestock + off_farm <= family + hired
    for t in range(T):
        for h in H:
            row = np.zeros(nvars)
            for cn in C:
                row[idx[('area', h, cn, t)]] = params.crops[cn].labor_req_per_ha
            for l in L:
                row[idx[('live_units', h, l, t)]] = params.livestock[l].labor_req_per_unit
            row[idx[('off_farm', h, None, t)]] = 1.0
            row[idx[('hired', h, None, t)]] = -1.0
            A_ub.append(row); b_ub.append(params.households[h].labor_endowment)

    # Hired cap
    for t in range(T):
        for h in H:
            row = np.zeros(nvars)
            row[idx[('hired', h, None, t)]] = 1.0
            A_ub.append(row); b_ub.append(params.households[h].max_hired_labor)

    # Calorie floor
    for t in range(T):
        for h in H:
            row = np.zeros(nvars)
            for cn in C:
                row[idx[('cons', h, cn, t)]] = -params.crops[cn].calorie_per_kg
            kcal_need = params.min_kcal_per_person_per_day * (params.households[h].adult_equiv * POPmul[t]) * params.days_per_year
            A_ub.append(row); b_ub.append(-kcal_need)

    res = linprog(cvec, A_ub=np.array(A_ub), b_ub=np.array(b_ub),
                  A_eq=np.array(A_eq), b_eq=np.array(b_eq),
                  bounds=bounds, method='highs')
    return res, idx, H, C, L


## Run solver and export yearly + NPV outputs

In [8]:
res, idx, H, C, L = solve_multi_year_lp(params, T, Ymul, Pmul, Wmul, Fmul, POPmul, DISC)
print('Status:', res.message)
print('Success:', bool(res.success))
if not res.success:
    raise RuntimeError('Optimization failed — check coefficients/bounds or calorie feasibility.')

x = res.x
def val(kind, h, k, t):
    return float(x[idx[(kind, h, k, t)]])

rows = []
for t in range(T):
    wage_t = params.prices.wage * Wmul[t]
    for h in H:
        rev_crops = sum(val('sold', h, cn, t) * (params.crops[cn].price_sale * Pmul[t]) for cn in C)
        rev_livestock = sum(val('live_units', h, l, t) * params.livestock[l].price_sale for l in L)
        rev_off = val('off_farm', h, None, t) * wage_t

        cost_crop_inputs = sum(val('area', h, cn, t) * (params.crops[cn].seed_cost_per_ha + params.crops[cn].fert_cost_per_ha * Fmul[t] + params.crops[cn].chem_cost_per_ha) for cn in C)
        cost_livestock = sum(val('live_units', h, l, t) * (params.livestock[l].feed_cost_per_unit + params.livestock[l].vet_cost_per_unit) for l in L)
        cost_hired = val('hired', h, None, t) * wage_t

        profit = (rev_crops + rev_livestock + rev_off) - (cost_crop_inputs + cost_livestock + cost_hired)
        rows.append({
            'year': t+1,
            'household_class': h,
            'rev_crops': rev_crops,
            'rev_livestock': rev_livestock,
            'rev_off_farm': rev_off,
            'cost_crop_inputs': cost_crop_inputs,
            'cost_livestock': cost_livestock,
            'cost_hired_labor': cost_hired,
            'profit': profit,
            'discount_factor': DISC[t],
            'discounted_profit': profit * DISC[t],
        })

yearly_df = pd.DataFrame(rows)
npv_df = (yearly_df.groupby('household_class', as_index=False)
          .agg(NPV=('discounted_profit','sum'),
               mean_profit=('profit','mean'),
               min_profit=('profit','min'),
               max_profit=('profit','max')))

if display_dataframe_to_user:
    display_dataframe_to_user('Phase 2A — Yearly results', yearly_df)
    display_dataframe_to_user('Phase 2A — NPV summary', npv_df)
else:
    display(yearly_df.head(12))
    display(npv_df)

yearly_df.to_csv(OUT/'phase2A_yearly_results.csv', index=False)
npv_df.to_csv(OUT/'phase2A_npv_summary.csv', index=False)
json.dump({'success': True, 'objective_npv': float(-res.fun), 'T': T, 'discount_rate': r}, open(OUT/'phase2A_meta.json','w'), indent=2)

print('Saved outputs in ./outputs/')


Status: Optimization terminated successfully. (HiGHS Status 7: Optimal)
Success: True


,year,household_class,rev_crops,rev_livestock,rev_off_farm,cost_crop_inputs,cost_livestock,cost_hired_labor,profit,discount_factor,discounted_profit
0,1,HFR,0.0,163395.238095,0.0,704.507937,40848.809524,9600.0,112241.920635,1.000000,112241.920635
1,1,HMR,0.0,178221.428571,0.0,723.047619,44555.357143,10400.0,122543.023810,1.000000,122543.023810
2,1,MFIR,0.0,222700.000000,0.0,778.666667,55675.000000,12800.0,153446.333333,1.000000,153446.333333
3,1,MMR,0.0,222700.000000,0.0,778.666667,55675.000000,12800.0,153446.333333,1.000000,153446.333333
4,1,MMIR,0.0,242004.761905,0.0,852.825397,60501.190476,14400.0,166250.746032,1.000000,166250.746032
5,1,MFR,0.0,208047.619048,0.0,741.587302,52011.904762,12000.0,143294.126984,1.000000,143294.126984
6,2,HFR,0.0,163395.238095,0.0,704.507937,40848.809524,9600.0,112241.920635,0.909091,102038.109668
7,2,HMR,0.0,178221.428571,0.0,723.047619,44555.357143,10400.0,122543.023810,0.909091,111402.748918
8,2,MFIR,0.0,222700.000000,0.0,778.666667,55675.000000,12800.0,153446.333333,0.909091,139496.666667
9,2,MMR,0.0,222700.000000,0.0,778.666667,55675.000000,12800.0,153446.333333,0.909091,139496.666667


,household_class,NPV,mean_profit,min_profit,max_profit
0,HFR,7.586458e+05,112241.920635,112241.920635,112241.920635
1,HMR,8.282712e+05,122543.023810,122543.023810,122543.023810
2,MFIR,1.037147e+06,153446.333333,153446.333333,153446.333333
3,MFR,9.685284e+05,143294.126984,143294.126984,143294.126984
4,MMIR,1.123693e+06,166250.746032,166250.746032,166250.746032
5,MMR,1.037147e+06,153446.333333,153446.333333,153446.333333


Saved outputs in ./outputs/


## Optional: Validate year 1 (production-only)
If `observed_prod_only.csv` exists, we compare year-1 model metrics to observed production-only metrics.

In [9]:
if obs_df is None:
    print('No observed_prod_only.csv found. Skipping validation.')
else:
    y1 = yearly_df[yearly_df['year']==1].copy().rename(columns={'rev_off_farm':'off_farm_labor'})
    metrics = ['rev_crops','rev_livestock','off_farm_labor','cost_crop_inputs','cost_livestock','cost_hired_labor']
    merged = y1.merge(obs_df[['household_class']+metrics], on='household_class', suffixes=('_model','_obs'), how='outer')

    def pct_diff(m,o):
        if pd.isna(m) or pd.isna(o): return np.nan
        if o==0: return np.inf if m!=0 else 0.0
        return 100.0*(m-o)/abs(o)

    comp_rows=[]
    for _, rrow in merged.iterrows():
        for m in metrics:
            comp_rows.append({
                'household_class': rrow['household_class'],
                'metric': m,
                'observed': rrow[f'{m}_obs'],
                'model': rrow[f'{m}_model'],
                'diff': rrow[f'{m}_model'] - rrow[f'{m}_obs'] if pd.notna(rrow[f'{m}_model']) and pd.notna(rrow[f'{m}_obs']) else np.nan,
                'pct_diff_%': pct_diff(rrow[f'{m}_model'], rrow[f'{m}_obs'])
            })
    comp_df = pd.DataFrame(comp_rows)

    agg=[]
    for m in metrics:
        sub = comp_df[comp_df['metric']==m].replace([np.inf,-np.inf], np.nan).dropna(subset=['observed','model'])
        rmse = float(np.sqrt(np.mean((sub['model']-sub['observed'])**2))) if len(sub)>0 else np.nan
        denom = sub['observed'].replace(0, np.nan)
        mape = float(np.mean(np.abs((sub['model']-sub['observed'])/denom))*100.0) if len(sub)>0 else np.nan
        agg.append({'metric': m, 'RMSE': rmse, 'MAPE_%': mape})
    agg_df = pd.DataFrame(agg)

    if display_dataframe_to_user:
        display_dataframe_to_user('Validation — Year 1 diffs', comp_df)
        display_dataframe_to_user('Validation — Year 1 summary', agg_df)
    else:
        display(comp_df)
        display(agg_df)

    comp_df.to_csv(OUT/'phase2A_validation_year1.csv', index=False)
    agg_df.to_csv(OUT/'phase2A_validation_year1_summary.csv', index=False)
    print('Saved validation outputs.')


,household_class,metric,observed,model,diff,pct_diff_%
0,HFR,rev_crops,1726.28,0.000000,-1726.280000,-100.000000
1,HFR,rev_livestock,844.47,163395.238095,162550.768095,19248.850533
2,HFR,off_farm_labor,463.33,0.000000,-463.330000,-100.000000
3,HFR,cost_crop_inputs,1166.30,704.507937,-461.792063,-39.594621
4,HFR,cost_livestock,231.00,40848.809524,40617.809524,17583.467326
5,HFR,cost_hired_labor,1195.30,9600.000000,8404.700000,703.145654
6,HMR,rev_crops,6104.32,0.000000,-6104.320000,-100.000000
7,HMR,rev_livestock,1568.32,178221.428571,176653.108571,11263.843385
8,HMR,off_farm_labor,3649.15,0.000000,-3649.150000,-100.000000
9,HMR,cost_crop_inputs,3061.10,723.047619,-2338.052381,-76.379484


,metric,RMSE,MAPE_%
0,rev_crops,10946.740093,100.000000
1,rev_livestock,204944.732339,12776.363890
2,off_farm_labor,2412.961644,100.000000
3,cost_crop_inputs,2023.106454,54.405793
4,cost_livestock,50894.021048,16324.007726
5,cost_hired_labor,11018.074616,1084.812196


Saved validation outputs.
